# Grid Search Table

In [7]:
# Packages
import pandas as pd
import joblib
import glob
import numpy as np
import dataframe_image as dfi # NOTE: YOU MUST HAVE GOOGLE CHROME INSTALLED FOR THIS TO WORK CORRECTLY

## Load Hyperparameter Settings

In [8]:
# Iterate over all files matching "Hyperparameter Settings.joblib" in subdirectories of '../../Output/Classifier Fitting'
list_of_files = []
list_of_settings = []
for file in glob.glob('../../Output/Classifier Fitting/**/*Hyperparameter Settings.joblib', recursive=True):
    list_of_files.append(file)
    list_of_settings.append(joblib.load(file))

print(list_of_settings)
print(list_of_files)

# list of classifier names - second to last item split on '\\'
classifier_names = [file.split('\\')[-2] for file in list_of_files]
print(classifier_names)

# pair list_of_settings and classifier_names
classifier_settings = dict(zip(classifier_names, list_of_settings))

[[{'solver': ['saga'], 'penalty': [None], 'C': [1], 'class_weight': [None, 'balanced'], 'multi_class': ['ovr', 'multinomial']}, {'solver': ['saga'], 'penalty': ['elasticnet'], 'C': [0.001, 0.01, 0.1, 1, 10, 100], 'l1_ratio': [0.0, 0.25, 0.5, 0.75, 1.0], 'class_weight': [None, 'balanced'], 'multi_class': ['ovr', 'multinomial']}], [{'solver': ['saga'], 'penalty': [None], 'C': [1], 'class_weight': [None, 'balanced'], 'multi_class': ['ovr', 'multinomial']}, {'solver': ['saga'], 'penalty': ['elasticnet'], 'C': [0.001, 0.01, 0.1, 1, 10, 100], 'l1_ratio': [0.0, 0.25, 0.5, 0.75, 1.0], 'class_weight': [None, 'balanced'], 'multi_class': ['ovr', 'multinomial']}], [{'solver': ['saga'], 'penalty': [None], 'C': [1], 'class_weight': [None, 'balanced'], 'multi_class': ['ovr', 'multinomial']}, {'solver': ['saga'], 'penalty': ['elasticnet'], 'C': [0.001, 0.01, 0.1, 1, 10, 100], 'l1_ratio': [0.0, 0.25, 0.5, 0.75, 1.0], 'class_weight': [None, 'balanced'], 'multi_class': ['ovr', 'multinomial']}], {'c_value

In [9]:
# print items in classifer_settings
for key, value in classifier_settings.items():
    print(key)
    print(value)

Logistic Regression
[{'solver': ['saga'], 'penalty': [None], 'C': [1], 'class_weight': [None, 'balanced'], 'multi_class': ['ovr', 'multinomial']}, {'solver': ['saga'], 'penalty': ['elasticnet'], 'C': [0.001, 0.01, 0.1, 1, 10, 100], 'l1_ratio': [0.0, 0.25, 0.5, 0.75, 1.0], 'class_weight': [None, 'balanced'], 'multi_class': ['ovr', 'multinomial']}]
SVM
{'c_values': array([1.e-01, 1.e+00, 1.e+01, 1.e+02]), 'kernel_grid': ['rbf', 'poly'], 'gamma_grid': array([1.e-01, 1.e+00, 1.e+01, 1.e+02]), 'degree_grid': [2, 3, 5, 7], 'k_folds': 5}
XGBoost
[{'learning_rate': [0.01, 0.1, 0.3], 'n_estimators': [100, 200], 'max_depth': [3, 6], 'subsample': [0.8, 1], 'colsample_bytree': [0.8, 1]}]


## Unnest Settings

In [10]:
# Lists for unnesting
classifiers = []
settings = []
settings_options = []
# Iterate over keys
for key in classifier_settings.keys():
    # If value is a list of dictionaries, combine into a single dictionary
    new_dict = {}
    if isinstance(classifier_settings[key], list):
        for d in classifier_settings[key]:
            # iterate over keys in dictionary
            for k in d.keys():
                # if key is in new_dict, append to list
                if k in new_dict.keys():
                    new_dict[k].append(d[k])
                # if key not in new_dict, create new list
                else:
                    new_dict[k] = [d[k]]
        classifier_settings[key] = new_dict
    # For each key in the unnested dictionary
    for k in classifier_settings[key].keys():
        # wrap any single values
        if not isinstance(classifier_settings[key][k], list) and not isinstance(classifier_settings[key][k], np.ndarray):
            classifier_settings[key][k] = [classifier_settings[key][k]]
        if isinstance(classifier_settings[key][k], np.ndarray):
            classifier_settings[key][k] = classifier_settings[key][k].tolist()
        # Append to lists
        classifiers.append(key)
        settings.append(k)
        settings_options.append(list(classifier_settings[key][k]))

## Dataframe of settings

In [11]:
# Put settings into a dataframe
classifier_settings_df = pd.DataFrame({'Classifier': classifiers, 'Setting': settings, 'Setting Options': settings_options})

# convert Setting Options to string
classifier_settings_df['Setting Options'] = classifier_settings_df['Setting Options'].apply(lambda x: ', '.join(map(str, x)))
# strip brackets
classifier_settings_df['Setting Options'] = classifier_settings_df['Setting Options'].str.replace('[', '')
classifier_settings_df['Setting Options'] = classifier_settings_df['Setting Options'].str.replace(']', '')
# expand out Setting Options by splitting on ',' and stacking
classifier_settings_df = classifier_settings_df.set_index(['Classifier', 'Setting'])['Setting Options'].str.split(', ', expand=True).stack().reset_index(level=2, drop=True).reset_index().rename(columns={0: 'Setting Options'}).drop_duplicates()
# strip '
classifier_settings_df['Setting Options'] = classifier_settings_df['Setting Options'].str.replace("'", '')
# recombined Setting Options by Classifer and Setting
classifier_settings_df['Setting Options'] = classifier_settings_df.groupby(['Classifier', 'Setting'])['Setting Options'].transform(lambda x: ', '.join(x))
# drop duplicates
classifier_settings_df = classifier_settings_df.drop_duplicates()
# Rename setting to Hyperparameter
classifier_settings_df = classifier_settings_df.rename(columns={'Setting': 'Hyperparameter'})
# Rename Setting Options to Options
classifier_settings_df = classifier_settings_df.rename(columns={'Setting Options': 'Options'})

classifier_settings_df

,Classifier,Hyperparameter,Options
0,Logistic Regression,solver,saga
2,Logistic Regression,penalty,"None, elasticnet"
4,Logistic Regression,C,"1, 0.001, 0.01, 0.1, 10, 100"
11,Logistic Regression,class_weight,"None, balanced"
15,Logistic Regression,multi_class,"ovr, multinomial"
19,Logistic Regression,l1_ratio,"0.0, 0.25, 0.5, 0.75, 1.0"
24,SVM,c_values,"0.09999999999999999, 1.0, 10.0, 100.0"
28,SVM,kernel_grid,"rbf, poly"
30,SVM,gamma_grid,"0.09999999999999999, 1.0, 10.0, 100.0"
34,SVM,degree_grid,"2, 3, 5, 7"


## Table Formatting

In [12]:
# Styled grid search output table
grid_search_table = (classifier_settings_df.style
                                        .format(thousands=",", decimal=".")
                                        .set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
                                        .set_properties(**{'text-align': 'center'})
                                        .hide(axis="index"))

# Save to PNG
dfi.export(grid_search_table, '../../Output/Classifier Fitting/Grid Search Table.png')

# Save to Excel
grid_search_table.to_excel('../../Output/Classifier Fitting/Grid Search Table.xlsx', index=False)

grid_search_table

Classifier,Hyperparameter,Options
Logistic Regression,solver,saga
Logistic Regression,penalty,"None, elasticnet"
Logistic Regression,C,"1, 0.001, 0.01, 0.1, 10, 100"
Logistic Regression,class_weight,"None, balanced"
Logistic Regression,multi_class,"ovr, multinomial"
Logistic Regression,l1_ratio,"0.0, 0.25, 0.5, 0.75, 1.0"
SVM,c_values,"0.09999999999999999, 1.0, 10.0, 100.0"
SVM,kernel_grid,"rbf, poly"
SVM,gamma_grid,"0.09999999999999999, 1.0, 10.0, 100.0"
SVM,degree_grid,"2, 3, 5, 7"
